#### sklearn.naive_bayes.GaussianNB

In [19]:
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]]) # Features
Y = np.array([1, 1, 1, 2, 2, 2]) # Labels
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X, Y)
GaussianNB(priors=None)
print(clf.predict([[-0.8, -1]]))

clf_pf = GaussianNB()
clf_pf.partial_fit(X, Y, np.unique(Y))
GaussianNB(priors=None)
print(clf_pf.predict([[-0.8, -1]]))

[1]
[1]


In [10]:
%run ./SourceFiles/ud120-projects/tools/startup.py


checking for nltk
checking for numpy
checking for scipy
checking for sklearn

downloading the Enron dataset (this may take a while)
to check on progress, you can cd up one level, then execute <ls -lthr>
Enron dataset should be last item on the list, along with its current size
download will complete at about 423 MB
download complete!

unzipping Enron dataset (this may take a while)
you're ready to go!


In [1]:
import os

for entry in os.scandir('.'):
    print(entry.name)

.git
.ipynb_checkpoints
data
FirstNotebook.ipynb
PimaPrediction.ipynb
python_ML_minicourse.ipynb
README.md
SourceFiles
Udacity.ipynb


### Changing lines in the file to fit Python 3 expectetations

In [2]:
original = "./SourceFiles/ud120-projects/tools/word_data.pkl"
destination = "./SourceFiles/ud120-projects/tools/word_data_unix.pkl"

content = ''
outsize = 0
with open(original, 'rb') as infile:
    content = infile.read()
with open(destination, 'wb') as output:
    for line in content.splitlines():
        outsize += len(line) + 1
        output.write(line + str.encode('\n'))

print("Done. Saved %s bytes." % (len(content)-outsize))

Done. Saved 35156 bytes.


### Pre-processing

In [3]:
import pickle
import _pickle as cPickle
import numpy

from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif



def preprocess(words_file = "./SourceFiles/ud120-projects/tools/word_data_unix.pkl", authors_file="./SourceFiles/ud120-projects/tools/email_authors.pkl"):
    """ 
        this function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) and performs
        a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features

        after this, the feaures and labels are put into numpy arrays, which play nice with sklearn functions

        4 objects are returned:
            -- training/testing features
            -- training/testing labels

    """

    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the text learning mini-project
    authors_file_handler = open(authors_file, 'rb')
    authors = pickle.load(authors_file_handler)
    authors_file_handler.close()

    words_file_handler = open(words_file, 'rb')
    word_data = cPickle.load(words_file_handler)
    words_file_handler.close()

    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)



    ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed  = vectorizer.transform(features_test)

    ### feature selection, because text is super high dimensional and 
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=10)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()

    ### info on the data
    print("no. of Chris training emails:", sum(labels_train))
    print("no. of Sara training emails:", len(labels_train)-sum(labels_train))
    
    return features_train_transformed, features_test_transformed, labels_train, labels_test


C:\Users\Aleks\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
features_train, features_test, labels_train, labels_test = preprocess()

no. of Chris training emails: 7936
no. of Sara training emails: 7884


### Actual training and prediction

In [6]:
from sklearn.naive_bayes import GaussianNB
from time import time
clf = GaussianNB()
t0 = time()
clf.fit(features_train, labels_train)
print("training time:", round(time()-t0, 3), "s")
GaussianNB(priors=None)
t0 = time()
pred = clf.predict(features_test)
print(pred)
print("prediction time:", round(time()-t0, 3), "s")

training time: 1.887 s
[0 0 1 ..., 1 0 0]
prediction time: 0.327 s


### Adding some metrics

In [7]:
import numpy as np
from sklearn.metrics import accuracy_score
acc = accuracy_score(labels_test, pred)
emailnum = numpy.sum(pred == 1)
print("Prediction accuracy: ", acc)
print("Chris's emails: ", emailnum)



Prediction accuracy:  0.973265073948
Chris's emails:  906


### Create tally of algorithm accuracy

In [16]:
import pandas
accuracy_tally_array= numpy.array([['GaussianNB', acc, emailnum]])
colnames = ['name', 'accuracy', 'chris emails']
accuracy_tally = pandas.DataFrame(accuracy_tally_array, columns= colnames)
print(accuracy_tally)

         name        accuracy chris emails
0  GaussianNB  0.973265073948          906


In [17]:
def displayAccuracyTally(accuracy_tally_array, array):
    accuracy_tally_array = np.vstack([accuracy_tally_array, array])
    colnames = ['name', 'accuracy', 'chris emails']
    accuracy_tally = pandas.DataFrame(accuracy_tally_array, columns= colnames)
    print(accuracy_tally)
    return accuracy_tally_array

## SVM

In [47]:
from sklearn import svm
X = [[0, 0], [1, 1]]
y = [0, 1]
clf = svm.SVC()
clf.fit(X, y)  
pred = clf.predict([[2., 2.]])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [48]:
clf.predict([[2., 2.]])

array([1])

In [21]:
from sklearn import svm
from time import time
def svmModeler(C, kernelType, features_train, features_test, labels_train, labels_test):
    clf = svm.SVC(kernel= kernelType, C= C)
    t0 = time()
    clf.fit(features_train, labels_train ) 
    print("training time:", round(time()-t0, 3), "s")

    t0 = time()
    pred = clf.predict(features_test)
    print(pred)
    print("prediction time:", round(time()-t0, 3), "s")

    from sklearn.metrics import accuracy_score
    acc = accuracy_score(labels_test, pred)
    emailnum = numpy.sum(pred == 1)
    print("Prediction accuracy: ", acc)
    row= numpy.array(['svm.SVC ' + kernelType + ' C=' + str(C) , acc, emailnum])
    global accuracy_tally_array
    accuracy_tally_array = displayAccuracyTally(accuracy_tally_array, row)

In [22]:
C = 1.0  # SVM regularization parameter
svmModeler(C, "linear", features_train, features_test, labels_train, labels_test)

training time: 278.551 s
[0 0 1 ..., 1 0 0]
prediction time: 29.702 s
Prediction accuracy:  0.984072810011
                   name        accuracy chris emails
0            GaussianNB  0.973265073948          906
1  svm.SVC linear C=1.0  0.984072810011          881


In [23]:
C = 2.0  # SVM regularization parameter
svmModeler(C, "linear", features_train, features_test, labels_train, labels_test)

training time: 227.418 s
[0 0 1 ..., 1 0 0]
prediction time: 23.392 s
Prediction accuracy:  0.988623435722
                   name        accuracy chris emails
0            GaussianNB  0.973265073948          906
1  svm.SVC linear C=1.0  0.984072810011          881
2  svm.SVC linear C=2.0  0.988623435722          877


In [24]:
C = 3.0  # SVM regularization parameter
svmModeler(C, "linear", features_train, features_test, labels_train, labels_test)

training time: 185.903 s
[0 0 1 ..., 1 0 0]
prediction time: 18.804 s
Prediction accuracy:  0.990898748578
                   name        accuracy chris emails
0            GaussianNB  0.973265073948          906
1  svm.SVC linear C=1.0  0.984072810011          881
2  svm.SVC linear C=2.0  0.988623435722          877
3  svm.SVC linear C=3.0  0.990898748578          877


In [25]:
C = 4.0  # SVM regularization parameter
svmModeler(C, "linear", features_train, features_test, labels_train, labels_test)

training time: 206.784 s
[0 0 1 ..., 1 0 0]
prediction time: 21.745 s
Prediction accuracy:  0.990329920364
                   name        accuracy chris emails
0            GaussianNB  0.973265073948          906
1  svm.SVC linear C=1.0  0.984072810011          881
2  svm.SVC linear C=2.0  0.988623435722          877
3  svm.SVC linear C=3.0  0.990898748578          877
4  svm.SVC linear C=4.0  0.990329920364          878


##### C=3 shows best accuracy 

#### Lets try smaller data set

In [42]:
sub_features_train = features_train[:np.int(len(features_train)/100)]
sub_labels_train = labels_train[:np.int(len(labels_train)/100)]
C = 1.0  # SVM regularization parameter
svmModeler(C, "linear", sub_features_train, features_test , sub_labels_train, labels_test)

training time: 0.119 s
[0 1 1 ..., 1 0 1]
prediction time: 1.186 s
Prediction accuracy:  0.884527872582


In [43]:
C = 2.0  # SVM regularization parameter
svmModeler(C, "linear", sub_features_train, features_test , sub_labels_train, labels_test)

training time: 0.12 s
[0 1 1 ..., 1 0 1]
prediction time: 1.09 s
Prediction accuracy:  0.895335608646


In [44]:
C = 3.0  # SVM regularization parameter
svmModeler(C, "linear", sub_features_train, features_test , sub_labels_train, labels_test)

training time: 0.118 s
[0 1 1 ..., 1 0 1]
prediction time: 1.107 s
Prediction accuracy:  0.900455062571


In [45]:
C = 4.0  # SVM regularization parameter
svmModeler(C, "linear", sub_features_train, features_test , sub_labels_train, labels_test)

training time: 0.118 s
[0 1 1 ..., 1 0 1]
prediction time: 1.054 s
Prediction accuracy:  0.89590443686


#### Lets try different kernel


In [47]:
C = 1.0  # SVM regularization parameter
svmModeler(C, "rbf", sub_features_train, features_test , sub_labels_train, labels_test)

training time: 0.13 s
[0 1 1 ..., 1 1 1]
prediction time: 1.28 s
Prediction accuracy:  0.616040955631


##### Not good at all

In [56]:
C = 2.0  # SVM regularization parameter
svmModeler(C, "rbf", sub_features_train, features_test , sub_labels_train, labels_test)

training time: 0.129 s
[0 1 1 ..., 1 1 1]
prediction time: 1.301 s
Prediction accuracy:  0.616040955631


In [57]:
C = 3.0  # SVM regularization parameter
svmModeler(C, "rbf", sub_features_train, features_test , sub_labels_train, labels_test)

training time: 0.132 s
[0 1 1 ..., 1 1 1]
prediction time: 1.328 s
Prediction accuracy:  0.616040955631


In [58]:
C = 4.0  # SVM regularization parameter
svmModeler(C, "rbf", sub_features_train, features_test , sub_labels_train, labels_test)

training time: 0.13 s
[0 1 1 ..., 1 1 1]
prediction time: 1.312 s
Prediction accuracy:  0.616040955631


In [59]:
C = 10.0  # SVM regularization parameter
svmModeler(C, "rbf", sub_features_train, features_test , sub_labels_train, labels_test)

training time: 0.13 s
[0 1 1 ..., 1 1 1]
prediction time: 1.302 s
Prediction accuracy:  0.616040955631


In [60]:
C = 100.0  # SVM regularization parameter
svmModeler(C, "rbf", sub_features_train, features_test , sub_labels_train, labels_test)

training time: 0.128 s
[0 1 1 ..., 1 1 1]
prediction time: 1.524 s
Prediction accuracy:  0.616040955631


In [61]:
C = 1000.0  # SVM regularization parameter
svmModeler(C, "rbf", sub_features_train, features_test , sub_labels_train, labels_test)

training time: 0.127 s
[0 1 1 ..., 1 0 1]
prediction time: 1.208 s
Prediction accuracy:  0.821387940842


In [80]:
C = 6000.0  # SVM regularization parameter
svmModeler(C, "rbf", sub_features_train, features_test , sub_labels_train, labels_test)

training time: 0.125 s
[0 1 1 ..., 1 0 1]
prediction time: 1.075 s
Prediction accuracy:  0.900455062571


In [85]:
C = 6000.0  # SVM regularization parameter
svmModeler(C, "rbf", features_train, features_test , labels_train, labels_test)

training time: 147.792 s
[0 0 1 ..., 1 0 0]
prediction time: 15.49 s
Prediction accuracy:  0.990898748578


In [100]:
print("Chris's emails: ", numpy.sum(feat == 1))

Chris's emails:  1014


## Decision Trees

In [135]:
from sklearn import tree
def decisionTreeSampler(min_samples_split, features_train, features_test, labels_train, labels_test):
    t0 = time()
    clf = tree.DecisionTreeClassifier(min_samples_split= min_samples_split)
    clf = clf.fit(features_train, labels_train)
    print("training time:", round(time()-t0, 3), "s")

    t0 = time()
    pred = clf.predict(features_test)
    print("prediction time:", round(time()-t0, 3), "s")

    from sklearn.metrics import accuracy_score
    print("Prediction accuracy: ", accuracy_score(labels_test, pred))
    print("Chris's emails: ", numpy.sum(pred == 1))

In [140]:
min_samples_split = 2
decisionTreeSampler(min_samples_split, features_train, features_test , labels_train, labels_test)

training time: 50.697 s
prediction time: 0.024 s
Prediction accuracy:  0.990898748578
Chris's emails:  863


In [141]:
min_samples_split = 3
decisionTreeSampler(min_samples_split, features_train, features_test , labels_train, labels_test)

training time: 48.503 s
prediction time: 0.023 s
Prediction accuracy:  0.992036405006
Chris's emails:  863


In [142]:
min_samples_split = 4
decisionTreeSampler(min_samples_split, features_train, features_test , labels_train, labels_test)

training time: 50.493 s
prediction time: 0.023 s
Prediction accuracy:  0.98976109215
Chris's emails:  861
